# Creating spreadsheet from B3 to income tax

# Criando uma planilha com dados retirados da B3 para declarar no impost do renda

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

Is necessary to put the data in raw_data directory because the informations are personas

Necessário colocar os dados em raw_data por se tratar de informações pessoais

In [2]:
path = '../raw_data'
files = [f for f in listdir(path) if isfile(join(path, f))] 

Feature engineering

Organizando as colunas 

In [3]:
list_files = []
for file in range(0, len(files)):
    list_file = pd.read_excel(f'{path}/{files[file]}', parse_dates=['Data'])
    list_files.append(list_file)
list_all_files = pd.concat(list_files)
#list_all_files.Data = pd.to_datetime(list_all_files.Data)
list_files_s = list_all_files.sort_values(by=['Produto', 'Data'])
df_files = pd.DataFrame(list_files_s)
df_files.dtypes

/home/thamirisbrandao/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Entrada/Saída                object
Data                 datetime64[ns]
Movimentação                 object
Produto                      object
Instituição                  object
Quantidade                   object
Preço unitário              float64
Valor da Operação           float64
dtype: object

In [5]:
df_files['Data da compra'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Credito')
df_files['Data da venda'] = df_files['Data'].where(df_files['Entrada/Saída'] == 'Debito')
df_files = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Movimentação', 'Instituição']]
df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
df_files['Quantidade'] = df_files['Quantidade'].astype(float)
df_files.dtypes

/tmp/ipykernel_756/492304967.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].apply(lambda x: x.replace(',','.'))
/tmp/ipykernel_756/492304967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_files['Quantidade'] = df_files['Quantidade'].astype(float)


Produto                      object
Data da compra       datetime64[ns]
Data da venda        datetime64[ns]
Quantidade                  float64
Preço unitário              float64
Valor da Operação           float64
Movimentação                 object
Instituição                  object
dtype: object

In [7]:
df_stock = df_files[df_files['Movimentação'] == 'Transferência - Liquidação']
df_stock = df_files[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']]
df_stock = df_stock.sort_values(by=['Produto']).reset_index()
df_stock

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição
0,48,ABEV3 - AMBEV S.A. ...,NaT,2020-05-19,100.0,11.81,1181.00,XP INVESTIMENTOS CCTVM S/A
1,6,AMER3 - B2W COMPANHIA DIGITAL,NaT,2021-10-11,72.0,32.67,2352.24,CLEAR CORRETORA - GRUPO XP
2,31,BBAS3 - BCO BRASIL S.A. ...,2020-03-06,NaT,50.0,31.86,1593.00,CLEAR CORRETORA - GRUPO XP
3,17,BBAS3 - BCO BRASIL S.A. ...,2020-06-24,NaT,50.0,33.70,1685.00,CLEAR CORRETORA - GRUPO XP
4,7,BBAS3 - BCO BRASIL S.A. ...,NaT,2021-05-24,210.0,32.83,6894.30,CLEAR CORRETORA - GRUPO XP
...,...,...,...,...,...,...,...,...
111,62,VVAR3 - VIA VAREJO S.A. ...,2020-03-20,NaT,900.0,4.35,3915.00,CLEAR CORRETORA - GRUPO XP
112,50,VVAR3 - VIA VAREJO S.A. ...,NaT,2020-05-05,1800.0,9.00,16200.00,CLEAR CORRETORA - GRUPO XP
113,41,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,2020-03-06,NaT,40.0,29.50,1180.00,CLEAR CORRETORA - GRUPO XP
114,14,YDUQ3 - YDUQS PARTICIPACOES S.A. ...,2020-07-28,NaT,40.0,32.98,1319.20,CLEAR CORRETORA - GRUPO XP


In [8]:
df_fixedi_c = df_files[df_files['Movimentação'] == 'Compra']
df_fixedi_v = df_files[df_files['Movimentação'] == 'Venda']
df_fixedi = pd.concat([df_fixedi_c, df_fixedi_v])
df_fixedi = df_fixedi[['Produto', 'Data da compra', 'Data da venda', 'Quantidade', 'Preço unitário', 'Valor da Operação', 'Instituição']]
df_fixedi = df_fixedi.sort_values(by=['Produto']).reset_index()
df_fixedi

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição
0,63,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A
1,78,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP
2,0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP
3,77,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP
4,6,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP


In [10]:
df_fixedi_buy = df_fixedi[df_fixedi['Data da compra'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).reset_index()
df_fixedi_buy

,Produto,Quantidade
0,Tesouro Selic 2025,6.86
1,Tesouro Selic 2027,7.34


In [42]:
df_fixedi_sel = df_fixedi[df_fixedi['Data da venda'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'}).transform(lambda x: x *-1).reset_index()
df_fixedi_sel

,Produto,Quantidade
0,Tesouro Selic 2023,-0.71
1,Tesouro Selic 2025,-2.14


In [89]:
t = pd.concat([df_fixedi_sel, df_fixedi_buy]).reset_index()
te = t.groupby(['Produto']).sum().reset_index()
for line in range(0, len(te)):
    if te.Quantidade[line] < 0:
        te.Quantidade[line] = te.Quantidade[line]*-1
    else:
        te.Quantidade[line]
te

/tmp/ipykernel_756/259721931.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te.Quantidade[line] = te.Quantidade[line]*-1


,Produto,index,Quantidade
0,Tesouro Selic 2023,0,0.71
1,Tesouro Selic 2025,1,4.72
2,Tesouro Selic 2027,1,7.34


In [90]:
te = pd.DataFrame(te)
te = te.drop(columns='index')
te

,Produto,Quantidade
0,Tesouro Selic 2023,0.71
1,Tesouro Selic 2025,4.72
2,Tesouro Selic 2027,7.34


In [91]:
te = te.rename(columns={'Quantidade': 'Saldo Total'})
te

,Produto,Saldo Total
0,Tesouro Selic 2023,0.71
1,Tesouro Selic 2025,4.72
2,Tesouro Selic 2027,7.34


In [ ]:
#Encontrar lucro ou prejuízo
#qual o tempo da venda?
#qual era o valor médio da ação no momento da venda?
#calcular venda-compra

#quantidade do produto da venda é igual quantidade do produto da compra? 
#se sim, calcular venda-compra

In [83]:
testando = pd.concat([df_fixedi, te]).reset_index()
testando

,level_0,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição,Total
0,0,63.0,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A,NaN
1,1,78.0,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP,NaN
2,2,0.0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP,NaN
3,3,77.0,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP,NaN
4,4,6.0,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP,NaN
5,0,NaN,Tesouro Selic 2023,NaT,NaT,NaN,NaN,NaN,NaN,0.71
6,1,NaN,Tesouro Selic 2025,NaT,NaT,NaN,NaN,NaN,NaN,4.72
7,2,NaN,Tesouro Selic 2027,NaT,NaT,NaN,NaN,NaN,NaN,7.34


In [9]:
duplicate = df_fixedi.Produto.duplicated()
duplicate

0    False
1    False
2     True
3     True
4    False
Name: Produto, dtype: bool

In [59]:
te.Quantidade[0] *-1

0.71

In [92]:
t = []
for q_b in range(0, len(df_fixedi_buy)):
    for q_s in range(0, len(df_fixedi_sel)):
        if df_fixedi_buy.Produto[q_b] == df_fixedi_sel.Produto[q_s]:
            quantity_b_s = df_fixedi_buy.Quantidade[q_b] - df_fixedi_sel.Quantidade[q_s]
            t.append(f'{df_fixedi_buy.Produto[q_b]} {quantity_b_s}')
t

['Tesouro Selic 2025 9.0']

In [93]:
print(f'{df_fixedi_buy.Produto[q_b]}')

Tesouro Selic 2027


In [94]:
df_fixedi['Quantidade Acumulada'] = df_fixedi['Quantidade'].cumsum()
df_fixedi

,index,Produto,Data da compra,Data da venda,Quantidade,Preço unitário,Valor da Operação,Instituição,Quantidade Acumulada
0,63,Tesouro Selic 2023,NaT,2020-03-17,0.71,10551.64,7491.66,XP INVESTIMENTOS CCTVM S/A,0.71
1,78,Tesouro Selic 2025,2020-07-01,NaT,2.14,10466.52,22398.35,CLEAR CORRETORA - GRUPO XP,2.85
2,0,Tesouro Selic 2025,2022-03-23,NaT,4.72,11457.89,54081.24,CLEAR CORRETORA - GRUPO XP,7.57
3,77,Tesouro Selic 2025,NaT,2020-03-02,2.14,10495.41,22460.18,CLEAR CORRETORA - GRUPO XP,9.71
4,6,Tesouro Selic 2027,2021-10-19,NaT,7.34,10905.72,80047.98,CLEAR CORRETORA - GRUPO XP,17.05


In [95]:
df_fixedi['Quantidade Acumulada'] = 0
for line in range(0, len(df_fixedi)):
    if df_fixedi['Produto'].iloc[line] != df_fixedi['Produto'].iloc[line+1]:
        df_fixedi['Quantidade Acumulada'].append(df_fixedi['Quantidade'])
df_fixedi

IndexError: single positional indexer is out-of-bounds

In [ ]:
df_fixedi_sel.Produto

In [ ]:
df_fixedi_buy = df_fixedi[df_fixedi['Data da compra'] > '20000101'].groupby('Produto').agg({'Quantidade':'sum'})


In [ ]:
df_fixedi_buy = df_fixedi_buy.reset_index()
df_fixedi_buy.Produto[0]

In [ ]:
#t = df_fixedi_buy.Quantidade[0] - df_fixedi_sel.Quantidade[1]
df_fixedi_buy[0]

In [ ]:
t

In [ ]:
df_fixedi_buy.merge(df_fixedi_sel, left_on='Quantidade', right_on='Quantidade')

In [ ]:
saldo_fixe = df_fixedi_buy.

In [ ]:
Saldo_de_acoes = df_fixedi.groupby('Produto').agg({'Quantidade':'sum'})
Saldo_de_acoes

In [ ]:
# create a lucro feature

In [ ]:
# Not work weel however was amazing
# df_fixedi['Saldo_de_acoes'] = ''
# for line in range(0, len(df_fixedi)):
#     if df_fixedi['Produto'].iloc[line] != df_fixedi['Produto'].iloc[line+1]:
#         df_fixedi['Saldo_de_acoes'].iloc[line] = Saldo_de_acoes['Quantidade'].iloc[0]
#     else:
#         df_fixedi['Saldo_de_acoes'].iloc[line] = 'nan'
#     if df_fixedi['Produto'].iloc[-1] == df_fixedi['Produto'].iloc[-2]:
#         df_fixedi['Saldo_de_acoes'].iloc[-1] = Saldo_de_acoes['Quantidade'].iloc[-1]

In [ ]:
df_fixedi.Quantidade[df_fixedi.Produto == df_fixedi.Produto].sum()

In [ ]:
df_fixedi

In [ ]:
#creating spreadsheet
#nome cnpj data da compra data da venda quantidade valor R$ total lucro ou prejuízo R$ saldo de de ações
b3_sheet = 